# Proyek Analisis Data: E-Commerce Public
- **Nama:** Rafael Simson Riston
- **Email:** rafaelsimsonriston@gmail.com
- **ID Dicoding:** rafaelsimsonr

# About Dataset

Brazilian E-Commerce Public Dataset by Olist

Welcome! This is a Brazilian ecommerce public dataset of orders made at Olist Store. The dataset has information of 100k orders from 2016 to 2018 made at multiple marketplaces in Brazil. Its features allows viewing an order from multiple dimensions: from order status, price, payment and freight performance to customer location, product attributes and finally reviews written by customers. We also released a geolocation dataset that relates Brazilian zip codes to lat/lng coordinates.

This is real commercial data, it has been anonymised, and references to the companies and partners in the review text have been replaced with the names of Game of Thrones great houses.

Context
This dataset was generously provided by Olist, the largest department store in Brazilian marketplaces. Olist connects small businesses from all over Brazil to channels without hassle and with a single contract. Those merchants are able to sell their products through the Olist Store and ship them directly to the customers using Olist logistics partners. See more on our website: www.olist.com

After a customer purchases the product from Olist Store a seller gets notified to fulfill that order. Once the customer receives the product, or the estimated delivery date is due, the customer gets a satisfaction survey by email where he can give a note for the purchase experience and write down some comments.

Attention
1. An order might have multiple items.
2. Each item might be fulfilled by a distinct seller.
3. All text identifying stores and partners where replaced by the names of Game of Thrones great houses.

# Menentukan Pertanyaan

1. sdfsd


# Import Packages

In [44]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_colwidth', None)

# Load Dataset

In [2]:
customers_df = pd.read_csv(os.path.join(os.getcwd(), 'data/customers_dataset.csv'))
customers_df.head() 

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [3]:
geo_df = pd.read_csv(os.path.join(os.getcwd(), 'data/geolocation_dataset.csv'))
geo_df.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [4]:
order_items_df = pd.read_csv(os.path.join(os.getcwd(), 'data/order_items_dataset.csv'))
order_items_df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [5]:
order_pay_df = pd.read_csv(os.path.join(os.getcwd(), 'data/order_payments_dataset.csv'))
order_pay_df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [6]:
order_reviews_df = pd.read_csv(os.path.join(os.getcwd(), 'data/order_reviews_dataset.csv'))
order_reviews_df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [7]:
orders_df = pd.read_csv(os.path.join(os.getcwd(), 'data/orders_dataset.csv'))
orders_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [8]:
product_category_name_df = pd.read_csv(os.path.join(os.getcwd(), 'data/product_category_name_translation.csv'))
product_category_name_df.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [9]:
products_df = pd.read_csv(os.path.join(os.getcwd(), 'data/products_dataset.csv'))
products_df.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [10]:
sellers_df = pd.read_csv(os.path.join(os.getcwd(), 'data/sellers_dataset.csv'))
sellers_df.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


# Explonatory Data Analysis

## 1. Explore a missing variabel

In [62]:
# Define names of dataframes
dataframe_names = ['sellers_df', 'products_df', 'product_category_name_df', 'orders_df', 'order_reviews_df', 'order_pay_df', 'order_items_df', 'geo_df', 'customers_df']

# List containing all dataframes
dataframes = [sellers_df, products_df, product_category_name_df, orders_df, order_reviews_df, order_pay_df, order_items_df, geo_df, customers_df]

# Create a dictionary to store check results
datas_check = {
    'data_name': [], 
    'n_rows': [], 
    'n_cols': [], 
    'sum_null': [], 
    'sum_col_null':[], 
    'name_col_null':[], 
    'sum_duplicated':[], 
    'sum_col_duplicate':[],
    'name_col_duplicate':[]
}

# Loop through dataframes and perform checks
for data_name, data in zip(dataframe_names, dataframes):
    datas_check['data_name'].append(data_name)
    datas_check['n_rows'].append(data.shape[0])
    datas_check['n_cols'].append(data.shape[1])
    datas_check['sum_null'].append(data.isna().sum().sum())
    datas_check['sum_duplicated'].append(data.duplicated().sum().sum())
    
    # Initialize lists for storing column-wise null and duplicate information
    sum_col_null = []
    name_col_null = []
    sum_col_duplicate = []
    name_col_duplicate = []
    
    # Loop through columns of each dataframe
    for col in data.columns:
        # Count null values and duplicates for each column
        sum_col_null.append(data[col].isna().sum())
        sum_col_duplicate.append(data.duplicated().sum().sum())
    
    # Count columns with null and duplicate values
    sum_col_n = sum(n != 0 for n in sum_col_null)
    sum_col_d = sum(n != 0 for n in sum_col_duplicate)
    
    # Append column names with null and duplicate values
    for idx, (n_null, n_duplicate) in enumerate(zip(sum_col_null, sum_col_duplicate)):
        if n_null != 0:
            name_col_null.append(data.columns[idx])
        if n_duplicate != 0:
            name_col_duplicate.append(data.columns[idx])

    datas_check['sum_col_null'].append(sum_col_n)
    datas_check['name_col_null'].append(name_col_null)
    datas_check['sum_col_duplicate'].append(sum_col_d)
    datas_check['name_col_duplicate'].append(name_col_duplicate)

# Convert dictionary to dataframe
summary_df = pd.DataFrame(datas_check)

In [63]:
summary_df

,data_name,n_rows,n_cols,sum_null,sum_col_null,name_col_null,sum_duplicated,sum_col_duplicate,name_col_duplicate
0,sellers_df,3095,4,0,0,[],0,0,[]
1,products_df,32951,9,2448,8,"[product_category_name, product_name_lenght, product_description_lenght, product_photos_qty, product_weight_g, product_length_cm, product_height_cm, product_width_cm]",0,0,[]
2,product_category_name_df,71,2,0,0,[],0,0,[]
3,orders_df,99441,8,4908,3,"[order_approved_at, order_delivered_carrier_date, order_delivered_customer_date]",0,0,[]
4,order_reviews_df,99224,7,145903,2,"[review_comment_title, review_comment_message]",0,0,[]
5,order_pay_df,103886,5,0,0,[],0,0,[]
6,order_items_df,112650,7,0,0,[],0,0,[]
7,geo_df,1000163,5,0,0,[],261831,5,"[geolocation_zip_code_prefix, geolocation_lat, geolocation_lng, geolocation_city, geolocation_state]"
8,customers_df,99441,5,0,0,[],0,0,[]


As you can see, there are a lot of missing values in the products dataset, orders, and reviews. On the other hand, duplicate values can be found in this data, specifically in the geographical dataset. However, this is normal because of the zip code, city, and state.

In [97]:
datas_describe = {
    'data_name':[],
    'column_name': [],
    'dtype': [],
    'count':[],
    'mean':[],
    'min':[],
    'max':[],
    'median':[]
}


for data_name, data in zip(dataframe_names, dataframes):
    for col in data.columns:
        datas_describe['data_name'].append(data_name)
        datas_describe['column_name'].append(col)
        datas_describe['dtype'].append(str(data[col].dtype))  # Convert dtype to string
        datas_describe['count'].append(data[col].count())  # Use count() for non-null values count
        datas_describe['mean'].append(data[col].mean() if data[col].dtype != 'object' else "")  # Calculate mean for numeric columns
        datas_describe['min'].append(data[col].min() if data[col].dtype != 'object' else "")  # Calculate min for numeric columns
        datas_describe['max'].append(data[col].max() if data[col].dtype != 'object' else "")  # Calculate max for numeric columns
        datas_describe['median'].append(data[col].median() if data[col].dtype != 'object' else "")  # Calculate median for numeric columns

pd.DataFrame(datas_describe)


,data_name,column_name,dtype,count,mean,min,max,median
0,sellers_df,seller_id,object,3095,,,,
1,sellers_df,seller_zip_code_prefix,int64,3095,32291.059451,1001,99730,14940.0
2,sellers_df,seller_city,object,3095,,,,
3,sellers_df,seller_state,object,3095,,,,
4,products_df,product_id,object,32951,,,,
5,products_df,product_category_name,object,32341,,,,
6,products_df,product_name_lenght,float64,32341,48.476949,5.0,76.0,51.0
7,products_df,product_description_lenght,float64,32341,771.495285,4.0,3992.0,595.0
8,products_df,product_photos_qty,float64,32341,2.188986,1.0,20.0,1.0
9,products_df,product_weight_g,float64,32949,2276.472488,0.0,40425.0,700.0
